In [2]:
!pip install -q langchain dspy-ai crewai google-search-results numexpr sentencepiece

^C
ERROR: Operation cancelled by user

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [11]:
%reload_ext autoreload
%autoreload 2

In [38]:
from langchain.agents import AgentExecutor, load_tools
from langchain.chat_models import ChatHuggingFace
from langchain_community.llms import HuggingFaceEndpoint
from langchain_core.tools import Tool
from scripts.modified_calculator import LLMMathChain
from scripts.agents import build_hf_agent, build_openai_agent
from scripts.run_agents import answer_questions

In [14]:
OPENAI_MODEL_ID = "gpt-3.5-turbo-0125"

# DSPy

In [89]:
import dspy

gpt4_llm_dspy = dspy.OpenAI(model=OPENAI_MODEL_ID)
dspy.settings.configure(lm=gpt4_llm_dspy)

### Define tools

In [93]:
llm = ChatOpenAI(mdoel=OPENAI_MODEL_ID)

langchain_math_tool = Tool(
    name="Calculator",
    description="Useful for when you need to answer questions about math.",
    func=LLMMathChain.from_llm(llm=llm).run,
    coroutine=LLMMathChain.from_llm(llm=llm).arun,
)
TOOLS = load_tools(["serpapi", "llm-math"], llm=llm)
langchain_serpapi_tool = TOOLS[0]


class DSPyMathTool:
    name = "calculator"
    input_variable = "math formula"
    desc = "Useful for when you need to answer questions about math."

    def __init__(self, k=3):
        pass

    def __call__(self, *args, **kwargs):
        return langchain_math_tool.run(*args, **kwargs)


class DSPySerpAPITool:
    name = "search"
    input_variable = "search input"
    desc = langchain_serpapi_tool.description

    def __init__(self, k=3):
        pass

    def __call__(self, *args, **kwargs):
        return langchain_serpapi_tool.run(*args, **kwargs)

In [94]:
import datetime
import dspy


class GiveTime:
    name = "GiveTime"
    input_variable = "empty"
    desc = "takes an empty string and returns the current local time"

    def __init__(self, k=3):
        pass

    def __call__(self, *args, **kwargs):
        return datetime.datetime.now().strftime("%H:%M:%S")

In [ ]:
math_tool = DSPyMathTool()
serpapi_tool = DSPySerpAPITool()
agent_dspy = dspy.ReAct("question -> answer", tools=[math_tool, serpapi_tool])

In [ ]:
async def call_dspy(agent, question: str) -> str:
    response = {"output": agent(question=question).answer, "intermediate_steps": []}
    return response

In [ ]:
await call_dspy(agent_dspy, "what is 2+2")

Backing off 0.1 seconds after 1 tries calling function <function GPT3.request at 0x7f1d8e981480> with kwargs {}
Backing off 1.1 seconds after 2 tries calling function <function GPT3.request at 0x7f1d8e981480> with kwargs {}
Backing off 0.8 seconds after 3 tries calling function <function GPT3.request at 0x7f1d8e981480> with kwargs {}
Backing off 7.6 seconds after 4 tries calling function <function GPT3.request at 0x7f1d8e981480> with kwargs {}
Backing off 9.7 seconds after 5 tries calling function <function GPT3.request at 0x7f1d8e981480> with kwargs {}
Backing off 28.2 seconds after 6 tries calling function <function GPT3.request at 0x7f1d8e981480> with kwargs {}
Backing off 28.1 seconds after 7 tries calling function <function GPT3.request at 0x7f1d8e981480> with kwargs {}
Backing off 90.6 seconds after 8 tries calling function <function GPT3.request at 0x7f1d8e981480> with kwargs {}
Backing off 93.6 seconds after 9 tries calling function <function GPT3.request at 0x7f1d8e981480> wit

### Compile DSPy

In [ ]:
import datasets

math_dataset = datasets.load_dataset("gsm8k", "main")["train"]


def build_dspy_trainset(dataset):
    dspy_dataset = []
    for i in range(len(dataset)):
        dspy_dataset.append(
            dspy.Example(
                question=dataset["question"][i],
                answer=dataset["answer"][i],
            ).with_inputs("question")
        )
    return dspy_dataset


# from dspy.teleprompt import BootstrapFewShot

# teleprompter = BootstrapFewShot(metric=validate_with_llm_judge, max_rounds=5)
# compiled_rag = teleprompter.compile(
#     agent,
#     trainset=build_dspy_trainset(math_dataset[:20]),
#     valset=build_dspy_trainset(math_dataset[20:50]),
# )

# Benchmark different frameworks

In [43]:
import datasets

math_dataset = (
    datasets.load_dataset("reasoning-machines/gsm-hard", "main")["train"]
    .shuffle(seed=42)
    .select(range(30))
)
math_dataset = math_dataset.rename_column("input", "question")
math_dataset = math_dataset.rename_column("target", "answer")

task_column = ["GSM8K-hard"] * len(math_dataset)
math_dataset = math_dataset.add_column("task", task_column).select_columns(
    ["question", "answer", "task"]
)
eval_dataset = math_dataset

OUTPUT_DIR = "output_gsm8k-hard"

/home/ubuntu/venv/agents/lib/python3.10/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by promote_options='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)


### DSPy

In [14]:
# DSPy
await answer_questions(
    eval_dataset,
    agent_dspy,
    "dspy_agent",
    output_folder=OUTPUT_DIR,
    agent_call_function=call_dspy,
)

NameError: name 'answer_questions' is not defined

### Vanilla Agent

In [28]:
prompt = "Answer the question below. You can think out loud, but remind to ALWAYS use the exact characters `Final Answer:` when you provide a definitive answer. Question:"


async def call_vanilla_agent(agent, question: str):
    output = agent.invoke(prompt + question).content
    if "Final Answer:" in output:
        output = output.split("Final Answer:")[1]
    response = {"output": output, "intermediate_steps": []}
    return response

In [29]:
from langchain.chat_models import ChatHuggingFace

In [31]:
endpoint("ok")

OKOKOK model::: None


', so I have had this gadget for about 6 months now. I must admit I was very excited when I bought it, as I have been a huge fan of "Chopped" on the Food Network for some time.\n\nWell, I have to say, I was somewhat disappointed. The "slicing discs" that come with it are really only good for thinly slicing soft items like potatoes and tomatoes. It\'s not so great at slicing anything else, like vegetables or meat. It seems to crush everything instead of slicing it, and the results are not pretty.\n\nI have also found that it is quite difficult to clean. All the nooks and crannies between the discs and the blade are hard to get into, and I always seem to end up with bits of food stuck in there that I can\'t seem to get out.\n\nAdditionally, the size of the gadget is a bit of a problem. It takes up a lot of counter space, and I have found that I really don\'t use it that often, so it\'s just kind of taking up space most of the time.\n\nOverall, I would say that this gadget is definitely n

In [45]:
MODEL_NAME = "zephyr-7b"
url = "https://gw38hkbdke51vu7l.us-east-1.aws.endpoints.huggingface.cloud"

In [ ]:
# llm_gpt = ChatOpenAI(model=MODEL_ID)
endpoint = HuggingFaceEndpoint(endpoint_url=url)
llm_agent = ChatHuggingFace(llm=endpoint)


await answer_questions(
    eval_dataset,
    llm_agent,
    f"{MODEL_NAME}_agent_notools",
    output_folder=OUTPUT_DIR,
    agent_call_function=call_vanilla_agent,
)

### LangChain

In [48]:
from scripts.agents import build_openai_agent, build_hf_agent

# langchain_agent = build_openai_agent(model_id=OPENAI_MODEL_ID)
langchain_agent = build_hf_agent(hf_endpoint_url=url)
await answer_questions(
    eval_dataset,
    langchain_agent,
    f"{MODEL_NAME}_langchain_nosearch",
    output_folder=OUTPUT_DIR,
)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/ubuntu/.cache/huggingface/token
Login successful
Removing search
Last message must be a HumanMessage!
System prompt not supported


  0%|          | 0/30 [00:00<?, ?it/s]



> Entering new AgentExecutor chain...
Invalid or incomplete response

### CrewAI

In [24]:
from crewai import Agent, Task, Crew, Process

langchain_math_tool = Tool(
    name="Calculator",
    description="Useful for when you need to answer questions about math.",
    func=LLMMathChain.from_llm(llm=llm_agent).run,
    coroutine=LLMMathChain.from_llm(llm=llm_agent).arun,
)
TOOLS = load_tools(["serpapi", "llm-math"], llm=llm_agent)
langchain_serpapi_tool = TOOLS[0]


async def call_crew_sequential(agent, question: str):
    # Creating a senior researcher agent
    researcher = Agent(
        role="Senior Researcher",
        goal=f"Solve this question: {question}",
        verbose=True,
        backstory="""Driven by curiosity, you like solving math problems""",
    )

    # Creating a writer agent
    veryfier = Agent(
        role="Senior Verifyer",
        goal=f"Check results of others for this question: {question}",
        verbose=True,
        backstory="""You have a flair to detect errors in math problems and correct them.""",
    )
    # Research task for identifying AI trends
    solving_task = Task(
        description=f"""Solve the given question: {question}. Think step-by-step.""",
        expected_output="A short answer",
        max_inter=3,
        tools=[langchain_math_tool, langchain_serpapi_tool],
        agent=researcher,
    )

    # Writing task based on research findings
    verification_task = Task(
        description=f"""You have to answer {question}, then provide a final answer.
        Think step-by-step to correct any errors. Remember to preface your final answer with 'Final Answer:'. In your final answer, do not talk about wether the previous answer was correct or not. Just provide the corrected answer.
        """,
        expected_output=f"The corrected answer to the question {question}",
        tools=[langchain_math_tool, langchain_serpapi_tool],
        agent=veryfier,
    )
    # Forming the tech-focused crew
    crew = Crew(
        agents=[researcher, veryfier],
        tasks=[solving_task, verification_task],
        process=Process.sequential,  # Sequential task execution
    )
    output = crew.kickoff()

    if "Final Answer:" in output:
        output = output.split("Final Answer:")[1]
    response = {"output": output, "intermediate_steps": []}
    return response


# llm_gpt = ChatOpenAI(model=OPENAI_MODEL_ID, temperature=0)

await answer_questions(
    eval_dataset,
    llm_agent,
    f"{MODEL_NAME}_crewai_sequential",
    output_folder=OUTPUT_DIR,
    agent_call_function=call_crew_sequential,
)

  0%|          | 0/30 [00:00<?, ?it/s]



> Entering new CrewAgentExecutor chain...
Thought: Firstly, let's figure out how many pounds of beeswax Charlie needs to make 1080379 candles, then we can calculate the cost of the supplies. After that, we can calculate the revenue from selling all these candles and finally subtract the cost from the revenue to get the net profit. To do these calculations, I will use the Calculator tool.

Action: Calculator

Action Input: First, let's calculate how many pounds of beeswax are needed to make 1080379 candles. If 1 pound makes 10 candles, then Charlie will need 1080379 / 10 = 108037.9 pounds of beeswax. The total cost for the beeswax and wicks will be 108037.9 * $10 = $1080379. The revenue from selling the candles will be 1080379 * $2 = $2160758. So, the net profit will be $2160758 - $1080379 = $1080379. 

I just used the Calculator tool with input {'tool_input': "First, let's calculate how many pounds of beeswax are needed to make 1080379 candles. If 1 pound makes 10 candles, then Charl

  3%|▎         | 1/30 [04:26<2:08:50, 266.56s/it]Overriding of current TracerProvider is not allowed




> Entering new CrewAgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: 
1. Total jury duty days: 3
2. Hours spent per day: 6
3. Total hours spent: 3 days * 6 hours/day = 18 hours
4. Daily pay: $557654
5. Total pay for 3 days: 3 days * $557654/day = $1672962
6. Total parking fee for 3 days: 3 * $3 = $9
7. Total earning after deducting parking fee: $1672962 - $9 = $1672953
8. Hourly earning: $1672953 / 18 hours = $92941.83 

I just used the Calculator tool with input {'tool_input': 'Total jury duty days: 3, Hours spent per day: 6, Total hours spent: 3 days * 6 hours/day = 18 hours, Daily pay: $557654, Total pay for 3 days: 3 days * $557654/day = $1672962, Total parking fee for 3 days: 3 * $3 = $9, Total earning after deducting parking fee: $1672962 - $9 = $1672953, Hourly earning: $1672953 / 18 hours = $92941.83'}. So I already know the result of that and don't need to use it again now. 
I could give my final answer if I'm ready, using exaclty 

  7%|▋         | 2/30 [06:46<1:29:36, 192.03s/it]Overriding of current TracerProvider is not allowed


...



> Entering new CrewAgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: The calculation for the total amount spent will be the cost of the books plus the cost of the pencils. So, the calculation will be (3 * 16) + (3 * 6). 

I just used the Calculator tool with input {'tool_input': '(3 * 16) + (3 * 6)'}. So I already know the result of that and don't need to use it again now. 
I could give my final answer if I'm ready, using exaclty the expected format bellow: 
```
Thought: Do I need to use a tool? No
Final Answer: [your response here]```


Do I need to use a tool? No
Final Answer: Ted spent a total of 66 dollars.

> Finished chain.


> Entering new CrewAgentExecutor chain...
Thought: Do I need to use a tool? No
Final Answer: Ted spent a total of 66 dollars.

> Finished chain.


 33%|███▎      | 10/30 [18:56<25:57, 77.85s/it]Overriding of current TracerProvider is not allowed




> Entering new CrewAgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: First, we need to find out how much of the tree can be turned into logs. This is 80% of 80 feet, which is 0.8*80 = 64 feet. Then, we divide 64 by 4 to find out how many 4-foot logs can be made from the tree, which is 64/4 = 16 logs. Each of these logs is turned into 5 planks, so we multiply 16 by 5 to find out how many planks in total, which is 16*5 = 80 planks. Finally, each plank is sold for $5128470, so we multiply 80 by 5128470 to find out the total money made, which is 80*5128470.Do I need to use a tool? Yes
Action: Calculator
Action Input: First, we need to find out how much of the tree can be turned into logs. This is 80% of 80 feet, which is 0.8*80. Then, we divide that result by 4 to find out how many 4-foot logs can be made from the tree. Each of these logs is turned into 5 planks, so we multiply the number of logs by 5 to find out how many planks in total. Final

 37%|███▋      | 11/30 [22:29<37:47, 119.32s/it]Overriding of current TracerProvider is not allowed




> Entering new CrewAgentExecutor chain...
Thought: Do I need to use a tool? No

Final Answer: Given that Mia is 40 years old and Emma is 16 years older, Emma is 56 years old. The average of their ages is the sum of their ages divided by 2, which is (40 + 56) / 2 = 48 years.

> Finished chain.


> Entering new CrewAgentExecutor chain...
Thought: Do I need to use a tool? No
Final Answer: The average of their ages is 48 years.

> Finished chain.


# Evaluate results

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from langchain.chat_models import ChatOpenAI

eval_chat_model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)
eval_model_name = "GPT4"

/home/ubuntu/venv/agents/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [23]:
import pandas as pd
import glob
import json

answer_file_path = "output_gsm8k-hard/answers.jsonl"

result_df = pd.concat([pd.read_json(f) for f in glob.glob("output_gsm8k-hard/*.json")])
result_df = result_df.drop(columns=["start_time", "end_time"])
result_df.to_json(answer_file_path, lines=True, orient="records")

In [24]:
answer_df = pd.read_json(answer_file_path, lines=True)
answers = answer_df.to_dict(orient="records")
answers = answers

In [25]:
answer_df["agent_name"].value_counts()

agent_name
mistral-7b_langchain            30
gpt3.5_langchain2               30
mistral-7b_agent_notools        30
gpt3.5_agent_notools            30
mistral-7b_crewai_sequential    11
gpt3.5_crewai_sequential         5
dspy_agent                       4
Name: count, dtype: int64

In [26]:
answers = answers[:80]

In [27]:
from scripts.evaluation import evaluate_answers
from scripts.prompts import EVALUATION_PROMPT_TEMPLATE

output_file_path = "output_gsm8k-hard/evaluation.jsonl"

results = await evaluate_answers(
    answers,
    eval_chat_model,
    "GPT4",
    EVALUATION_PROMPT_TEMPLATE,
    output_file_path=output_file_path,
)

print("Evaluation is complete!")

Previous evaluations:
Launching evaluation for 40 examples...


  0%|          | 0/40 [00:00<?, ?it/s]

Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example


100%|██████████| 40/40 [00:30<00:00,  1.30it/s]

Evaluation is complete!


In [31]:
answer_df.iloc[:80]["agent_name"].value_counts()

agent_name
mistral-7b_langchain            30
mistral-7b_agent_notools        30
mistral-7b_crewai_sequential    11
gpt3.5_crewai_sequential         5
dspy_agent                       4
Name: count, dtype: int64

In [32]:
results_df = pd.read_json(output_file_path, lines=True)
results_df["agent_name"].value_counts()

agent_name
mistral-7b_langchain            30
mistral-7b_agent_notools        30
mistral-7b_crewai_sequential    11
gpt3.5_crewai_sequential         5
dspy_agent                       4
Name: count, dtype: int64

In [33]:
results_df["eval_score_GPT4"] = results_df["eval_score_GPT4"].astype(float)
results_df["eval_score_GPT4"] = (results_df["eval_score_GPT4"] - 1) / 4
display(results_df.groupby("agent_name")["eval_score_GPT4"].mean())

agent_name
dspy_agent                      0.000000
gpt3.5_crewai_sequential        0.950000
mistral-7b_agent_notools        0.158333
mistral-7b_crewai_sequential    0.590909
mistral-7b_langchain            0.066667
Name: eval_score_GPT4, dtype: float64

##### Thoughts on DSPy:


# CrewAI